In [9]:
!pip install pycaret pandas shap


[notice] A new release of pip is available: 24.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
from pycaret.regression import *
import datetime

# Load cleaned dataset
df = pd.read_csv("/content/cleaned_pm2_5_cleaned.csv")

# Create hour_group feature
def categorize_hour(hour):
    if 5 <= hour < 11:
        return "Morning"
    elif 11 <= hour < 17:
        return "Afternoon"
    elif 17 <= hour < 21:
        return "Evening"
    else:
        return "Night"

df["hour_group"] = df["hour"].apply(categorize_hour)

# Setup PyCaret environment
exp = setup(df, target='pm_2_5', session_id=123, train_size=0.8, categorical_features=['hour_group', 'weekday'])

# Train a Random Forest model
model = create_model('rf')

def predict_next_7_days(model, df):
    last_date = datetime.datetime.now()
    future_dates = [last_date + datetime.timedelta(days=i) for i in range(1, 8)]

    future_df = pd.DataFrame({
        'humidity': [df['humidity'].mean()] * 7,
        'temperature': [df['temperature'].mean()] * 7,
        'hour': [12] * 7,  # Assume noon for predictions
        'day': [d.day for d in future_dates],
        'month': [d.month for d in future_dates],
        'weekday': [(d.weekday()) for d in future_dates],  # Add weekday feature
        'hour_group': ['Afternoon'] * 7  # Default to 'Afternoon'
    })

    predictions = predict_model(model, data=future_df)
    future_df['pm_2_5_prediction'] = predictions.iloc[:, -1]  # Extract prediction column
    future_df['date'] = future_dates

    return future_df[['date', 'pm_2_5_prediction']]

# Predict next 7 days
future_predictions = predict_next_7_days(model, df)
print(future_predictions)

# Evaluate model
metrics = pull()
print("Model Performance:")
print(metrics)


RuntimeError: ('Pycaret only supports python 3.9, 3.10, 3.11. Your actual Python version: ', sys.version_info(major=3, minor=12, micro=4, releaselevel='final', serial=0), 'Please DOWNGRADE your Python version.')